# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None #to ignore the warning about my chained hotel_df
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
filepath = "../output_data/cities_random.csv"
cities_df = pd.read_csv(filepath)
cities_df.head()

,City_Id,Country,City,Date,Lat,Long,Cloudiness,Humidity,Wind_Speed,Temp_Max
0,3838859,AR,Río Gallegos,1601341191,-51.62,-69.22,20,69,4.70,35.6
1,3833367,AR,Ushuaia,1601341101,-54.80,-68.30,20,92,10.29,32.0
2,3874787,CL,Punta Arenas,1601341093,-53.15,-70.92,20,55,8.05,33.8
3,4845585,US,Westport,1601341175,41.14,-73.36,1,83,5.82,69.8
4,3874930,CL,Puerto Varas,1601341268,-41.32,-72.99,90,93,4.70,48.2


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key = g_key)

In [4]:
#Store the values needed to create the map
locations = cities_df[["Lat","Long"]]
humidity = cities_df["Humidity"]
#Plot the heatmap
fig = gmaps.figure()
#create and add layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                dissipating = False, point_radius = 3,
                                 max_intensity = max(cities_df["Lat"]))
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows with null values.

In [5]:
hotel_df = cities_df[(cities_df["Cloudiness"]==0)& 
                             (cities_df["Temp_Max"]>=70)&(cities_df["Temp_Max"]<80)&
                             (cities_df["Wind_Speed"]<10)&
                             (cities_df["Humidity"]<60)&(cities_df["Humidity"]>20)]
hotel_df

,City_Id,Country,City,Date,Lat,Long,Cloudiness,Humidity,Wind_Speed,Temp_Max
145,3456445,BR,Mutum,1601341396,-19.80,-41.44,0,50,3.00,78.91
163,3457398,BR,Mato Verde,1601341414,-15.40,-42.87,0,24,9.10,76.51
190,3388145,BR,São Raimundo Nonato,1601341438,-9.02,-42.70,0,34,0.27,76.39
323,1279023,IN,Alīganj,1601341569,27.50,79.18,0,55,2.46,75.88
335,1272653,IN,Dibai,1601341581,28.22,78.25,0,41,2.95,74.46
363,1279403,IN,Abohar,1601341611,30.15,74.18,0,32,0.98,75.20
369,2266272,PT,Moncarapacho,1601341615,37.09,-7.79,0,49,6.93,71.60
376,145459,IR,Ābādān,1601341621,30.34,48.30,0,41,2.24,77.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df.loc[:,'Hotel Name']= ''
hotel_df = hotel_df.reset_index(drop = True)
hotel_df

,City_Id,Country,City,Date,Lat,Long,Cloudiness,Humidity,Wind_Speed,Temp_Max,Hotel Name
0,3456445,BR,Mutum,1601341396,-19.80,-41.44,0,50,3.00,78.91,
1,3457398,BR,Mato Verde,1601341414,-15.40,-42.87,0,24,9.10,76.51,
2,3388145,BR,São Raimundo Nonato,1601341438,-9.02,-42.70,0,34,0.27,76.39,
3,1279023,IN,Alīganj,1601341569,27.50,79.18,0,55,2.46,75.88,
4,1272653,IN,Dibai,1601341581,28.22,78.25,0,41,2.95,74.46,
5,1279403,IN,Abohar,1601341611,30.15,74.18,0,32,0.98,75.20,
6,2266272,PT,Moncarapacho,1601341615,37.09,-7.79,0,49,6.93,71.60,
7,145459,IR,Ābādān,1601341621,30.34,48.30,0,41,2.24,77.00,


In [7]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
params = {
    "key" : g_key,
    "keyword" : "hotel",
    "radius" : 5000,
    "rankby" : "prominence",
}

for index, row in hotel_df.iterrows():

    # get lat and lng from df
    lat = row['Lat']
    lng = row["Long"]

    # add location to params
    params['location'] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    results = response["results"]
    try:
        print(f"The most prominent hotel in {row['City']} {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Retrieving Results for Index 0: Mutum.
The most prominent hotel in Mutum STELA HOTEL.
Retrieving Results for Index 1: Mato Verde.
The most prominent hotel in Mato Verde Quality Hotel.
Retrieving Results for Index 2: São Raimundo Nonato.
The most prominent hotel in São Raimundo Nonato Real Hotel.
Retrieving Results for Index 3: Alīganj.
Missing field/result... skipping.
Retrieving Results for Index 4: Dibai.
Missing field/result... skipping.
Retrieving Results for Index 5: Abohar.
The most prominent hotel in Abohar Toor Hotel.
Retrieving Results for Index 6: Moncarapacho.
The most prominent hotel in Moncarapacho Vila Monte Farm House.
Retrieving Results for Index 7: Ābādān.
The most prominent hotel in Ābādān Hotel Pars.


In [8]:
hotel_df.head()

,City_Id,Country,City,Date,Lat,Long,Cloudiness,Humidity,Wind_Speed,Temp_Max,Hotel Name
0,3456445,BR,Mutum,1601341396,-19.80,-41.44,0,50,3.00,78.91,STELA HOTEL
1,3457398,BR,Mato Verde,1601341414,-15.40,-42.87,0,24,9.10,76.51,Quality Hotel
2,3388145,BR,São Raimundo Nonato,1601341438,-9.02,-42.70,0,34,0.27,76.39,Real Hotel
3,1279023,IN,Alīganj,1601341569,27.50,79.18,0,55,2.46,75.88,
4,1272653,IN,Dibai,1601341581,28.22,78.25,0,41,2.95,74.46,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [10]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations,info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))